# Projeto 1 - Ciência dos Dados

Nome: Mateus Marinheiro

Nome: Henrique Albuquerque


Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [15]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

In [16]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
c:\Users\Mateus Marinhero\Documents\Insper\2023\2o Semestre\Ciência dos Dados\Projeto1\Projeto-1-C-Dados


Carregando a base de dados com os tweets classificados manualmente:

In [17]:
train = pd.read_excel('dados_treino.xlsx')
train.head(5)

,Mensagem,Target
0,Fique atento na compra. Ainda está com problem...,Relevante
1,Talvez a tradução antiga tivesse um pouco mais...,Relevante
2,"Sinceramente, também estou tentando entender c...",Relevante
3,Escolhi o livro por acreditar no tema e indica...,Irrelevante
4,"Recebi o produto avariado, no mesmo dia reclam...",Relevante


In [18]:
train.Target.value_counts()

Irrelevante    153
Relevante      147
Name: Target, dtype: int64

In [19]:
test = pd.read_excel('dados_teste.xlsx')
test.head(5)

,Mensagem,Target
0,"Não sou de desistir de livro, mas neste não te...",Irrelevante
1,EU NEM SE QUER RECEBI O PRODUTO! Voces são mui...,Relevante
2,Tao ruim quanto o primeiro. Fonte não justific...,Relevante
3,"Sou fã do canal, estava na primeira sessão da ...",Irrelevante
4,Péssimo. O autor claramente tem um visão disto...,Irrelevante


In [20]:
test.Target.value_counts()

Irrelevante    162
Relevante       38
Name: Target, dtype: int64

___
## Classificador automático de feedbacks de produtos da Amazon


Será criado um classificador segundo a teoria de probabilidade de Naïve Bayes, classificando notícias atuais sobre o mercado digital como:

- Relevante: Mensagens consideradas como relevantes serão aqueles que trazem alguma crítica à direcionadas a Amazon, sejam elas, em relação à preços, envios e outros problemas. Também serão considerados relevantes críticas feitas às editoras, sendo assim, mensagens criticando a tradução, formatação do texto, qualidade do produto entre outros.</li>
    
- Irrelevante: Mensagens consideradas como Irrelevantes serão aquelas que trazem qualquer tipo de opinião sobre o livro, qualquer tipo de elogio e qualquer outra coisa que não se enquadra como relevante.  </li>
    

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

#### 1 - Funções de  _cleanup_

Função que irá retirar das mensagens recebidas todos os ruídos, sendo eles, pountuações e símbolos, e também retiramos emojis, e números, que foram considerados irrelevantes para o nosso classificador.

In [28]:
# https://docs.python.org/3/library/re.html#
import re 


def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    #import string
    punctuation = '[´"!-.:?;$''/\|]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return text_subbed

import re

def remove_emojis(text):
    # Define um padrão de regex para encontrar emojis
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # Emoticons
                               u"\U0001F300-\U0001F5FF"  # Símbolos e pictogramas diversos
                               u"\U0001F680-\U0001F6FF"  # Transporte e símbolos de mapa
                               u"\U0001F700-\U0001F77F"  # Símbolos de alquimia, religião e zodíaco
                               u"\U0001F780-\U0001F7FF"  # Símbolos de traços diversos
                               u"\U0001F800-\U0001F8FF"  # Símbolos de escrita linear abugida
                               u"\U0001F900-\U0001F9FF"  # Símbolos suplementares de escrita linear abugida
                               u"\U0001FA00-\U0001FA6F"  # Símbolos de xadrez
                               u"\U0001FA70-\U0001FAFF"  # Símbolos de emojis adicionais
                               u"\U0001F004-\U0001F0CF"  # Símbolos de cartas de baralho
                               u"\U0001F170-\U0001F251"  # Símbolos de emojis de comemoração
                               "]+", flags=re.UNICODE)

    # Substitui os emojis por uma string vazia
    clean_text = emoji_pattern.sub(r'', text)

    return clean_text



#### 2 - Limpeza dos comentários

In [30]:
palavras_relevantes = []             #Lista com as palvras de feedbacks relevantes.
palavras_irrelevantes = []         #Lista com as palvras de feedbacks irrelevantes.

train_relevantes = train.loc[train.Target == 'Relevante', :]        #base de dados relevante
train_irrelevantes = train.loc[train.Target == 'Irrelevante', :]        #base de dados irrelevantes

for i in range(len(train_relevantes)):      #Limpando os emojis e potuação da base de dados relevante e colocando as palavras em uma lista.
    palavras_relevantes += cleanup(remove_emojis(train_relevantes.iloc[i, 0].lower())).split()

for i in range(len(train_irrelevantes)):    #Limpando os emojis e pontuação da base de dados irrelevante e colocando as palavras em uma lista.
    palavras_irrelevantes += cleanup(remove_emojis(train_irrelevantes.iloc[i, 0].lower())).split()

#Tirando númreros das listas de palavras.

for i in range(len(palavras_relevantes)):
    for caractere in palavras_relevantes[i]:
        if caractere.isdigit():
            palavras_relevantes[i] = ''
    
for i in range(len(palavras_irrelevantes)):
    for caractere in palavras_irrelevantes[i]:
        if caractere.isdigit():
            palavras_irrelevantes[i] = ''

#Criando lista com todas as palavras
total_palavras = palavras_relevantes + palavras_irrelevantes


#### 3 - Criando pd.series para cada Target

In [23]:
#Criando Série para palavras relevantes
serie_relevantes = pd.Series(palavras_relevantes)

#Criando Série para palavras não relevantes
serie_irrelevantes = pd.Series(palavras_irrelevantes)

#Criando série para todas as palavras:
serie_total = pd.Series(total_palavras)

#### 4 - Gerando tabela de frequências para as Séries

In [24]:
#Tabela de frequência de palavras relevantes
tabela_relevante = serie_relevantes.value_counts()

#Tabela de frequência de palavras não relevantes
tabela_irrelevante = serie_irrelevantes.value_counts()

#Tabela de frequência de todas as palavras
tabela_total = serie_total.value_counts()


___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

#### 1 - Comentários da base de  dados dos testes

In [36]:
palavras_test = []                #Lista para armazenar os feedbacks de teste.
palavras_relevantes_test = []      #Listas para armazenar as palavras relevantes dos feedbacks de teste.
palavras_irrelevantes_test = []    #Listas para armazenar as palavras irrelevantes dos feedbacks de teste.

test_relevantes = test.loc[train.Target == 'Relevante', :]        #base de dados de teste relevante
test_irrelevantes = test.loc[train.Target == 'Irrelevante', :]        #base de dados de teste irrelevantes

for i in range(len(test_relevantes)):      #Limpando os emojis e potuação da base de dados de teste relevante e colocando as palavras em uma lista.
    palavras_relevantes_test += cleanup(remove_emojis(test_relevantes.iloc[i, 0].lower())).split()

for i in range(len(test_irrelevantes)):    #Limpando os emojis e pontuação da base de dados de teste irrelevante e colocando as palavras em uma lista.
    palavras_irrelevantes_test += cleanup(remove_emojis(test_irrelevantes.iloc[i, 0].lower())).split()

#Tirando númreros das listas de palavras.

for i in range(len(palavras_relevantes_test)):
    for caractere in palavras_relevantes_test[i]:
        if caractere.isdigit():
            palavras_relevantes_test[i] = ''
    
for i in range(len(palavras_irrelevantes_test)):
    for caractere in palavras_irrelevantes_test[i]:
        if caractere.isdigit():
            palavras_irrelevantes_test[i] = ''

#Criando lista com todas as palavras da base de teste
total_palavras_test = palavras_relevantes_test + palavras_irrelevantes_test


<div id="probabilidades"></div>

## Probabilidades

Relembrando, queremos descobrir, dentre outras coisas, a probabilidade de uma frase ocorrer para um dos dois targets. Para isso, vamos assumir uma simplificação **bastante ingênua** de que a probabilidade de uma palavra da frase ocorrer em uma obra é independente da probabilidade de qualquer outra palavra. Assim, assumindo independência, **a probabilidade da frase é igual ao produto das probabilidades de cada palavra individualmente**.

Vamos assumir para fins destas análises que **a frequência relativa observada nestes textos é igual à probabilidade**. Ou seja, que se quisermos encontrar $P(portugal|Lusíadas)$ basta checar a frequência da palavra `'portugal'`: `tabela_lusiadas_relativa["portugal"]` (note que o acesso é semelhante ao acesso a um dicionário).

Ou seja, a $P(portugal|Lusíadas)$  pode ser obtida buscando a frequência relativa da palavra `portugal` dentro da tabela do texto Os Lusíadas:

___
### Concluindo

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nos tweets. Ex: stemming, lemmatization, stopwords
* CONSIDEROU mais de duas categorias na variável Target e INCREMENTOU a quantidade de notícias, mantendo pelo menos 250 notícias por categoria (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* Para Target com duas categorias: CRIOU pelo menos quatro categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto (pelo menos dois cenários, exceto aqueles já apresentados em sala pelos professores: por exemplo, filtro de spam)
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item Qualidade do Classificador a partir de novas separações das Notícias entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**